In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install pandas==1.5.2

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import pickle
from sklearn.metrics import f1_score

In [ ]:
def convert_sparse_matrix_to_sparse_tensor(X):
  coo = X.tocoo()
  indices = np.mat([coo.row, coo.col]).transpose()
  return tf.SparseTensor(indices, coo.data, coo.shape)

In [ ]:
with open('/content/drive/My Drive/MTS ML CUP/train/age/X_df_val_age_1_count.pickle', 'rb') as f:
  X_count_val = pickle.load(f)
with open('/content/drive/My Drive/MTS ML CUP/train/age/X_df_val_age_1_city_name.pickle', 'rb') as f:
  X_city_name_val = pickle.load(f)
with open('/content/drive/My Drive/MTS ML CUP/train/age/X_df_val_age_1_cpe_manufacturer_name.pickle', 'rb') as f:
  X_cpe_manufacturer_name_val = pickle.load(f)
with open('/content/drive/My Drive/MTS ML CUP/train/age/X_df_val_age_1_cpe_model_name.pickle', 'rb') as f:
  X_cpe_model_name_val = pickle.load(f)
with open('/content/drive/My Drive/MTS ML CUP/train/age/X_df_val_age_1_cpe_model_os_type.pickle', 'rb') as f:
  X_cpe_model_os_type_val = pickle.load(f)
with open('/content/drive/My Drive/MTS ML CUP/train/age/X_df_val_age_1_cpe_type_cd.pickle', 'rb') as f:
  X_cpe_type_cd_val = pickle.load(f)
with open('/content/drive/My Drive/MTS ML CUP/train/age/X_df_val_age_1_region_name.pickle', 'rb') as f:
  X_region_name_val = pickle.load(f)
with open('/content/drive/My Drive/MTS ML CUP/train/age/X_df_val_age_1_text.pickle', 'rb') as f:
  X_text_val = pickle.load(f)
with open('/content/drive/My Drive/MTS ML CUP/train/age/X_df_val_age_1_url_host.pickle', 'rb') as f:
  X_url_host_val = pickle.load(f)
# загружаем файл векторизованный tf-idf
with open('/content/drive/My Drive/MTS ML CUP/train/age/X_df_val_age_1_url_host_all.pickle', 'rb') as f:
  X_url_host_all_val = pickle.load(f)
with open('/content/drive/My Drive/MTS ML CUP/train/age/y_val_age_1.pickle', 'rb') as f:
  y_val = pickle.load(f)

In [ ]:
X_count_val = np.array(X_count_val)
X_city_name_val = convert_sparse_matrix_to_sparse_tensor(X_city_name_val)
X_cpe_manufacturer_name_val = convert_sparse_matrix_to_sparse_tensor(X_cpe_manufacturer_name_val)
X_cpe_model_name_val = convert_sparse_matrix_to_sparse_tensor(X_cpe_model_name_val)
X_cpe_model_os_type_val = convert_sparse_matrix_to_sparse_tensor(X_cpe_model_os_type_val)
X_cpe_type_cd_val = convert_sparse_matrix_to_sparse_tensor(X_cpe_type_cd_val)
X_region_name_val = convert_sparse_matrix_to_sparse_tensor(X_region_name_val)
X_text_val = convert_sparse_matrix_to_sparse_tensor(X_text_val)
X_url_host_val = convert_sparse_matrix_to_sparse_tensor(X_url_host_val)
X_url_host_val = tf.sparse.reorder(sp_input=X_url_host_val, name=None)
#X_url_host_all_val = convert_sparse_matrix_to_sparse_tensor(X_url_host_all_val)
#X_url_host_all_val = tf.sparse.reorder(sp_input=X_url_host_all_val, name=None)
y_val = np.array(y_val)

In [ ]:
model_discret = tf.keras.models.load_model("/content/drive/My Drive/MTS ML CUP/age_1_discret_19-71.keras")
model_continuos = tf.keras.models.load_model("/content/drive/My Drive/MTS ML CUP/age_1_continuos.keras")

In [ ]:
y_val_predict_discret = model_discret.predict([X_count_val,
                                       X_city_name_val,
                                       X_cpe_manufacturer_name_val,
                                       X_cpe_model_name_val,
                                       X_cpe_model_os_type_val,
                                       X_cpe_type_cd_val,
                                       X_region_name_val,
                                       X_text_val,
                                       X_url_host_val,
                                       X_url_host_all_val])

In [ ]:
y_val_predict_continuos = model_continuos.predict([X_count_val,
                                       X_city_name_val,
                                       X_cpe_manufacturer_name_val,
                                       X_cpe_model_name_val,
                                       X_cpe_model_os_type_val,
                                       X_cpe_type_cd_val,
                                       X_region_name_val,
                                       X_text_val,
                                       X_url_host_val,
                                       X_url_host_all_val])

In [ ]:
labels = np.array([19., 20., 21., 22., 23., 24., 25., 26., 27., 28., 29., 30.,
                   31., 32., 33., 34., 35., 36., 37., 38., 39., 40., 41., 42.,
                   43., 44., 45., 46., 47., 48., 49., 50., 51., 52., 53., 54.,
                   55., 56., 57., 58., 59., 60., 61., 62., 63., 64., 65., 66.,
                   67., 68., 69., 70., 71.])
y_val_predict_dis = np.zeros(len(y_val_predict_discret))
for i, y in enumerate(y_val_predict_discret):
  lab = labels[y / y.max() > 0.85]
  y_val_predict_dis[i] = lab.mean()

In [ ]:
y_val_predict_dis = (y_val_predict_dis + y_val_predict_continuos.flatten()) / 2

Класс 1 —19-25, Класс 2 —26-35, Класс 3 —36-45, Класс 4 —46-55, Класс 5 —56-65, Класс 6— 66+

In [ ]:
y_val_score = []
for x in y_val:
  if x <= 25:
    y_val_score.append(1)
  elif 25 < x <= 35:
    y_val_score.append(2)
  elif 35 < x <= 45:
    y_val_score.append(3)
  elif 45 < x <= 55:
    y_val_score.append(4)
  elif 55 < x <= 65:
    y_val_score.append(5)
  elif 65 < x:
    y_val_score.append(6)
y_val_score = np.array(y_val_score)

In [ ]:
y_init = [25, 35, 45, 55, 65]
y_val_predict_score = []
for x in y_val_predict_dis:
  if x <= y_init[0]:
    y_val_predict_score.append(1)
  elif y_init[0] < x <= y_init[1]:
    y_val_predict_score.append(2)
  elif y_init[1] < x <= y_init[2]:
    y_val_predict_score.append(3)
  elif y_init[2] < x <= y_init[3]:
    y_val_predict_score.append(4)
  elif y_init[3] < x <= y_init[4]:
    y_val_predict_score.append(5)
  elif y_init[4] < x:
    y_val_predict_score.append(6)
y_val_predict_score = np.array(y_val_predict_score)
f1 = f1_score(y_val_score, y_val_predict_score, average='weighted')
for i in range(5):
  f1_max = 0
  while f1_max < f1_score(y_val_score, y_val_predict_score, average='weighted'):
    f1_max = f1_score(y_val_score, y_val_predict_score, average='weighted')
    y_init[i] += 0.1
    y_val_predict_score = []
    for x in y_val_predict_dis:
      if x <= y_init[0]:
        y_val_predict_score.append(1)
      elif y_init[0] < x <= y_init[1]:
        y_val_predict_score.append(2)
      elif y_init[1] < x <= y_init[2]:
        y_val_predict_score.append(3)
      elif y_init[2] < x <= y_init[3]:
        y_val_predict_score.append(4)
      elif y_init[3] < x <= y_init[4]:
        y_val_predict_score.append(5)
      elif y_init[4] < x:
        y_val_predict_score.append(6)
    y_val_predict_score = np.array(y_val_predict_score)
  y_init[i] -= 0.1

In [ ]:
y_init

[26.50000000000002, 35.50000000000001, 45.0, 55.1, 65.1]